In [1]:
#for test
#https://medium.com/dropout-analytics/selenium-and-geckodriver-on-mac-b411dbfe61bc 참고

from time import sleep
from selenium import webdriver

story = 'https://medium.com/dropout-analytics/selenium-and-geckodriver-on-mac-b411dbfe61bc'
story = story + '?source=friends_link&sk=18e2c2f07fbe1f8ae53fef5ad57dbb12'   # 'https://bit.ly/2WaKraO' <- short link

def gecko_test(site_000=story):
    """
    simple overview:
        1) set up webdriver
        2) load this article 
        3) close up shop 
    
    input:
        >> site_000
            > default: url of this article ('friend link')
    """
    # set the driver 
    driver = webdriver.Firefox()

    # load this article 
    driver.get(site_000)
    # and chill a bit
    sleep(7)

    # k, cool. let's bounce. 
    driver.quit()


# make runable 
if __name__ == '__main__':
    # here we go
    gecko_test()

In [3]:
#imports here
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver import FirefoxOptions


from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time

import numpy as np
import pandas as pd
f = open("new_hashtags.csv", "r") # input file containing list of hashtags
out_dir = "./data/" # output directory for the scraped .csv files


options = FirefoxOptions()
options.add_argument('--no-sandbox')
#options.add_argument("--headless")
options.add_argument('--disable-dev-shm-usage')

#driver = webdriver.FireFox(service=Service(ChromeDriverManager().install()), options=options)
driver = webdriver.Firefox(options=options)

#driver.get("https://www.google.com")
#print(driver.title)
#driver.close()


#open the webpage
driver.get("http://www.instagram.com")
print("[+] Opened Instagram")

#target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))


#sonchirayi
#instagramak@80

#enter username and password
username.clear()
user = "USERNAME"
username.send_keys(user)
password.clear()
password.send_keys("PASSWORD")
time.sleep(5)
#target the login button and click it
button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
print(f"[+] Logging into Instagram (username: {user})...")

# handle alerts
time.sleep(10)
#alert = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Not Now")]'))).click()

alert = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.x1i10hfl'))).click()


print("[+] Logged in!")
print("[+] 1st popup clicked")
time.sleep(5)

alert2 = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button._a9--:nth-child(2)'))).click()
print("2nd popup clicked")


STOP_AT = 1 # stop at this number of hashtags
count = 0
for line in f:
    h = line.strip("#\n")
    count = count + 1
    print(f"[{count}] Scraping keyword: #{h}...")
    out_file = f"{out_dir}{h}.csv"


    # search for a certain hashtag
    time.sleep(5)
    print("    [+] Targetting the searchbox for keyword...")
    button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable(
        (By.XPATH, "//*[@aria-label='Search']"))).click()

    #target the search input field
    searchbox = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Search']")))
    searchbox.clear()

    #search for the hashtag
    keyword = f"#{h}"
    searchbox.send_keys(keyword)

    #FIXING THE DOUBLE ENTER
    time.sleep(5)  # Wait for 5 seconds
    my_link = WebDriverWait(driver, 15).until(EC.element_to_be_clickable(
        (By.XPATH, "//a[contains(@href, '/" + keyword[1:] + "/')]")))
    my_link.click()

    print("    [+] Waiting for page to load...")
    time.sleep(25)

    #increase the range to scroll more
    n_scrolls = 4
    print("    [+] Scrolling...")
    for j in range(0, n_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(25)

    #target all the link elements on the page
    anchors = driver.find_elements(By.TAG_NAME,'a')
    anchors = [a.get_attribute('href') for a in anchors]
    #narrow down all links to image links only
    anchors = [a for a in anchors if str(
        a).startswith("https://www.instagram.com/p/")]

    print('    [+] Found ' + str(len(anchors)) + ' links to posts')

    df = pd.DataFrame(np.array(anchors))
    print("    [+] Writing to csv...")
    df.to_csv(out_file, header = None, index=False)
    print("    [-] Done!")
    if(count==STOP_AT):
        break

driver.close()


"""
# search for a certain hashtag
#target the search button and click it
print("[+] Targetting the searchbox for keyword...")
button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH,"//*[@aria-label='Search']"))).click()

#target the search input field
searchbox = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Search']")))
#searchbox = EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Search']"))
searchbox.clear()

#search for the hashtag cat
keyword = "#cat"
searchbox.send_keys(keyword)
print(f"[INFO] Searching for {keyword}...")
 
#FIXING THE DOUBLE ENTER
time.sleep(5) # Wait for 5 seconds
my_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, '/" + keyword[1:] + "/')]")))
my_link.click()

print("[+] Waiting for page to load...")
time.sleep(25)

#increase the range to sroll more
n_scrolls = 4
print("[+] Scrolling...")
for j in range(0, n_scrolls):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(20)

#target all the link elements on the page
anchors = driver.find_elements(By.TAG_NAME,'a')
anchors = [a.get_attribute('href') for a in anchors]
#narrow down all links to image links only
anchors = [a for a in anchors if str(a).startswith("https://www.instagram.com/p/")]

print('Found ' + str(len(anchors)) + ' links to posts')
print(anchors[:5])

driver.close()
"""

[+] Opened Instagram
[+] Logging into Instagram (username: lookwhatitook_sw)...
[+] Logged in!
[+] 1st popup clicked
2nd popup clicked
[1] Scraping keyword: #vapecommunity...
    [+] Targetting the searchbox for keyword...
    [+] Waiting for page to load...
    [+] Scrolling...
    [+] Found 30 links to posts
    [+] Writing to csv...
    [-] Done!


'\n# search for a certain hashtag\n#target the search button and click it\nprint("[+] Targetting the searchbox for keyword...")\nbutton = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH,"//*[@aria-label=\'Search\']"))).click()\n\n#target the search input field\nsearchbox = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder=\'Search\']")))\n#searchbox = EC.element_to_be_clickable((By.XPATH, "//input[@placeholder=\'Search\']"))\nsearchbox.clear()\n\n#search for the hashtag cat\nkeyword = "#cat"\nsearchbox.send_keys(keyword)\nprint(f"[INFO] Searching for {keyword}...")\n \n#FIXING THE DOUBLE ENTER\ntime.sleep(5) # Wait for 5 seconds\nmy_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, \'/" + keyword[1:] + "/\')]")))\nmy_link.click()\n\nprint("[+] Waiting for page to load...")\ntime.sleep(25)\n\n#increase the range to sroll more\nn_scrolls = 4\nprint("[+] Scrolling...")\nfor j in 

In [10]:
# code to strip shortcodes from csv files in post_links
# store in scrapedData by folders
# ------------Run this in the scraped data folder directly, it will automatically create folders----------

from instaloader import Instaloader, Post
import os

in_dir = "data" #directory containing .csv files of extracted post-links
dir_path = os.getcwd()
in_dir = os.path.join(dir_path, in_dir)


L = Instaloader()
username = 'USERNAME'
password = 'PASSWORD'
L.login(username, password)

"""
# Create directory if it doesn't exist to save images
def create_folder(image_path):
    CHECK_FOLDER = os.path.isdir(image_path)
    # If folder doesn't exist, then create it.
    if not CHECK_FOLDER:
        os.makedirs(image_path)
"""

for file in os.listdir(in_dir):
    # get filename and open file
    file_path = os.path.join(in_dir, file)
    file_name = os.path.splitext(file)
    file_name = file_name[0]
    f = open(file_path,"r")
    print(f"[+] Opened {file}...")
    
    # create folder for the corresponding .csv file
    #data = os.path.join(out_dir, f"{file_name}")
    #create_folder(data)

    for line in f:
        p = line.strip("\n")
        p = p.split("https://www.instagram.com/p/")
        shortCode = p[1]
        shortCode = shortCode.strip("/")

        post = Post.from_shortcode(L.context, shortCode)
        L.download_post(post, target = file_name)

[+] Opened vapecommunity.csv...
vapecommunity/2023-06-01_22-58-33_UTC.jpg [• #teamnoct] comments json 
vapecommunity/2023-06-01_15-26-23_UTC_1.webp vapecommunity/2023-06-01_15-26-23_UTC_2.jpg vapecommunity/2023-06-01_15-26-23_UTC_2.mp4 vapecommunity/2023-06-01_15-26-23_UTC_3.jpg vapecommunity/2023-06-01_15-26-23_UTC_3.mp4 vapecommunity/2023-06-01_15-26-23_UTC_4.jpg vapecommunity/2023-06-01_15-26-23_UTC_4.mp4 [Uma receita muito maravilhosa…] 

KeyError: 'graphql'

In [16]:
import ujson as json
import pandas as pd

records = map(json.loads, open('zeeschuimer-export-tiktok.com-2023-06-02T053241.ndjson'))
df = pd.DataFrame.from_records(records)
df

df2 = pd.json_normalize(df['data'])
df2.columns

Index(['id', 'desc', 'createTime', 'challenges', 'originalItem', 'officalItem',
       'textExtra', 'secret', 'forFriend', 'digged', 'itemCommentStatus',
       'showNotPass', 'vl1', 'itemMute', 'privateItem', 'duetEnabled',
       'stitchEnabled', 'shareEnabled', 'isAd', 'collected', 'video.id',
       'video.height', 'video.width', 'video.duration', 'video.ratio',
       'video.cover', 'video.originCover', 'video.dynamicCover',
       'video.playAddr', 'video.downloadAddr', 'video.shareCover',
       'video.reflowCover', 'video.bitrate', 'video.encodedType',
       'video.format', 'video.videoQuality', 'video.encodeUserTag',
       'author.id', 'author.uniqueId', 'author.nickname', 'author.avatarThumb',
       'author.avatarMedium', 'author.avatarLarger', 'author.signature',
       'author.verified', 'author.secUid', 'author.secret', 'author.ftc',
       'author.relation', 'author.openFavorite', 'author.commentSetting',
       'author.duetSetting', 'author.stitchSetting', 'author.pri

In [19]:
import ujson as json
import pandas as pd

records = map(json.loads, open('zeeschuimer-export-instagram.com-2023-06-02T045424.ndjson'))
df = pd.DataFrame.from_records(records)
df

df2 = pd.json_normalize(df['data'])
df
#df2.columns

,nav_index,item_id,timestamp_collected,source_platform,source_platform_url,source_url,user_agent,data,id
0,1:12:NaN,3114033376067018178_29761405213,1685681637329,instagram.com,https://www.instagram.com/explore/tags/vapecom...,https://www.instagram.com/api/v1/feed/timeline/,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; ...,"{'taken_at': 1685441728, 'pk': '31140333760670...",30
1,1:12:NaN,3114027692801863574_1381562409,1685681637330,instagram.com,https://www.instagram.com/explore/tags/vapecom...,https://www.instagram.com/api/v1/feed/timeline/,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; ...,"{'taken_at': 1685441050, 'pk': '31140276928018...",31
2,1:12:NaN,3114608489854208912_1976585520,1685681637331,instagram.com,https://www.instagram.com/explore/tags/vapecom...,https://www.instagram.com/api/v1/feed/timeline/,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; ...,"{'taken_at': 1685510287, 'pk': '31146084898542...",32
3,1:12:NaN,3115867004636734951_10097797605,1685681637504,instagram.com,https://www.instagram.com/explore/tags/vapecom...,https://www.instagram.com/api/v1/tags/web_info...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; ...,"{'taken_at': 1685660313, 'pk': '31158670046367...",33
4,1:12:NaN,3116039133176042803_37827162199,1685681637504,instagram.com,https://www.instagram.com/explore/tags/vapecom...,https://www.instagram.com/api/v1/tags/web_info...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; ...,"{'taken_at': 1685680833, 'pk': '31160391331760...",34
...,...,...,...,...,...,...,...,...,...
62,1:12:NaN,3115658284660482589_37705474505,1685681661793,instagram.com,https://www.instagram.com/explore/tags/vapecom...,https://www.instagram.com/api/v1/tags/vapecomm...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; ...,"{'taken_at': 1685635432, 'pk': '31156582846604...",92
63,1:12:NaN,3115656392231363667_2234303672,1685681661794,instagram.com,https://www.instagram.com/explore/tags/vapecom...,https://www.instagram.com/api/v1/tags/vapecomm...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; ...,"{'taken_at': 1685635207, 'pk': '31156563922313...",93
64,1:12:NaN,3115655958783658201_58845895184,1685681661794,instagram.com,https://www.instagram.com/explore/tags/vapecom...,https://www.instagram.com/api/v1/tags/vapecomm...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; ...,"{'taken_at': 1685635155, 'pk': '31156559587836...",94
65,1:12:NaN,3115652328740575124_32031387474,1685681661794,instagram.com,https://www.instagram.com/explore/tags/vapecom...,https://www.instagram.com/api/v1/tags/vapecomm...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; ...,"{'taken_at': 1685634722, 'pk': '31156523287405...",95


In [1]:
import instaloader

# download pictures, download json file, but don't compress json file.
L = instaloader.Instaloader(download_pictures=True,
                            download_video_thumbnails=True,
                            download_videos=False,
                            download_geotags=True,
                            download_comments=True,
                            save_metadata=True,
                            compress_json=False
                            )

L.login('USERNAME', 'PASSWORD')


# two arguments: keyword and number of files
def get_it(keyword,numbers_of_files):
    while numbers_of_files > 0:
        for post in L.get_hashtag_posts(keyword):
            # post is an instance of instaloader.Post
            L.download_post(post, target='#'+keyword)
            numbers_of_files-=1
            # print(numbers_of_files)
            if numbers_of_files ==0:
                break
        print("loop ended for " + keyword)

# sample function
get_it("elfbar",100)

KeyboardInterrupt: 

### json to csv

In [112]:
import csv
import os
import json
import datetime

hashtag= '#puffbar'
path = './data/Instagram/{}/'.format(hashtag) 

fileList = os.listdir(path)
jobList = [] 
hashTagList = [] 
resultList=[] 

for file in fileList:
    if os.path.splitext(file)[1] == '.txt' : 
        jobList.append(os.path.splitext(file)[0])

report = './data/Instagram_report/report_{}_'.format(hashtag) + datetime.datetime.now().strftime('%Y%m%d') + '.txt'
f = open(report, 'w', encoding='utf-8', newline='')
for file in jobList:
    resultList.clear()
    if file+'.json' in fileList :
        with open(path + file+'.json') as post:
            post = json.load(post)

            id = post['node']['id']
            likeCount = post['node']['edge_liked_by']['count']
            commentCount = post['node']['edge_media_to_comment']['count']
            text = ' '.join(post['node']['edge_media_to_caption']['edges'][0]['node']['text'].split()) 

            hashTagList.clear()

            for idx, val in enumerate(text):
                tempstr = ''
                if val =='#':
                    location = idx
                    while(True):
                        tempstr += text[location]
                        location += 1
                        try:
                            if text[location] == ' ' or text[location] == '#' or text[location] == '\n':
                                break
                        except:
                            break
                    hashTagList.append(tempstr.replace("#",""))

            resultList.append(id)
            resultList.append(likeCount)
            resultList.append(commentCount)
            resultList.append(text)
            resultList.append(file)
            resultList.append(hashTagList.copy())

            wr = csv.writer(f)
            wr.writerow(resultList)
f.close()